# Problem description

Hi, FlORence ici 🇫🇷

I'm a manager of AmazOR that wants to find the best possible locations for our warehouses while deciding how to allocate our client demands to these warehouses.

Our warehouses can only handle a limited amount of demand but we need to ensure that all client demands are met.

While opening a new facility, we have some fixed costs, and we also take into account the cost of allocating the demand of each client to each warehouse.

Of course, we want to minimize our costs here...

Can you help me solve this problem?

# Problem data

Here are the first 32 lines of the provided data file:

    # The first line gives you the number of warehouses and the number of clients
    # The following <number of warehouses> lines give you the maximum demand a warehouse can handle and the fixed cost for opening that location
    # The rest of the lines give you:
    # - the demand for a client
    # - the cost for handling that demand to each warehouse

    # In this instance, you have 16 warehouses and 50 clients
    # Then, for each warehouse we have a maximum demand of 5000 and a fixed cost of 7500 (except for one warehouse)
    # After that, the demand of client 1 is 146, and the cost for handling that demand in warehouse 1 is 6739.72500, in warehouse 2 is 10355.05000, ... in warehouse 16 is 6051.70000
    # The demand of client 2 is 87, and the cost for handling that demand in warehouse 1 is 3204.86250, in warehouse 2 is 5457.07500, ... in warehouse 16 is 2838.37500
    # and so on and so forth

     16 50
     5000 7500.
     5000 7500.
     5000 7500.
     5000 7500.
     5000 7500.
     5000 7500.
     5000 7500.
     5000 7500.
     5000 7500.
     5000 7500.
     5000 0.
     5000 7500.
     5000 7500.
     5000 7500.
     5000 7500.
     5000 7500.
     146
     6739.72500 10355.05000 7650.40000 5219.50000 5776.12500 6641.17500 4374.52500
     3847.10000 6429.47500 5396.52500 5219.50000 4182.90000 7391.25000 5038.82500
     10349.57500 6051.70000

# Solution

The 34th client has a demand of 12912, which by far exceeds any warehouse. Thus, a single client's demand package cannot be atomic. We must consider the "cost per demand" instead.

We provide a simple greedy algorithm to distribute the clients' demands across warehouses. Iterative optimization algorithms may improve the cost.

In [2]:
from collections.abc import Iterable
from typing import NamedTuple

import numpy as np


class ProblemSpec(NamedTuple):
    n_warehouses: int
    n_clients: int

    warehouse_capacity: np.ndarray
    warehouse_base_cost: np.ndarray

    client_demand: np.ndarray
    per_client_cost: np.ndarray


def iter_data_file_numbers(path: str) -> Iterable[int | float]:
    with open(path) as file:
        for row in file:
            if row.startswith('#'):
                continue

            for num_str in row.split():
                try:
                    yield int(num_str)
                except ValueError:
                    yield float(num_str)


def read_data(path: str) -> ProblemSpec:
    it = iter(iter_data_file_numbers(path))

    n_warehouses = next(it)
    assert isinstance(n_warehouses, int)

    n_clients = next(it)
    assert isinstance(n_clients, int)

    warehouse_capacity = np.empty((n_warehouses,), dtype=int)
    warehouse_base_cost = np.empty((n_warehouses,), dtype=float)

    for j in range(n_warehouses):
        demand = next(it)
        assert isinstance(demand, int)
        warehouse_capacity[j] = demand

        cost = next(it)
        assert isinstance(cost, float)
        warehouse_base_cost[j] = cost

    client_demand = np.empty((n_clients), dtype=int)
    per_client_cost = np.empty((n_clients, n_warehouses), dtype=float)

    for i in range(n_clients):
        demand = next(it)
        assert isinstance(demand, int)
        client_demand[i] = demand

        for j in range(n_warehouses):
            cost = next(it)
            assert isinstance(cost, float)
            per_client_cost[i, j] = cost

    return ProblemSpec(
        n_warehouses,
        n_clients,
        warehouse_capacity,
        warehouse_base_cost,
        client_demand,
        per_client_cost
    )


def is_solution_feasible(spec: ProblemSpec, mtx: np.ndarray) -> bool:
    for i in range(spec.n_clients):
        if sum(mtx[i, :]) != spec.client_demand[i]:
            return False

    return all(
        sum(mtx[:, j]) <= spec.warehouse_capacity[j]
        for j in range(spec.n_warehouses)
    )


def objective_function(spec: ProblemSpec, mtx: np.ndarray) -> float:
    if not is_solution_feasible(spec, mtx):
        return float('inf')

    total = 0.0

    for j in range(spec.n_warehouses):
        cost_for_warehouse = sum(mtx[:, j] * spec.per_client_cost[:, j] / spec.client_demand)

        if cost_for_warehouse > 0.0:
            total += spec.warehouse_base_cost[j] + cost_for_warehouse

    return float(total)


def get_greedy_solution(spec: ProblemSpec) -> np.ndarray:
    sol = np.empty((spec.n_clients, spec.n_warehouses))
    capacity = spec.warehouse_capacity.copy()
    demand = spec.client_demand.copy()

    for i in range(spec.n_clients):
        # Warehouse ids sorted by cost
        by_cost = sorted(
            range(spec.n_warehouses),
            key=lambda j: spec.warehouse_base_cost[j] + spec.per_client_cost[i, j] / spec.client_demand[i]
        )

        for j in by_cost:
            sol[i, j] = min(capacity[j], demand[i])
            capacity[j] -= sol[i, j]
            demand[i] -= sol[i, j]

    return sol


spec = read_data('data/05.txt')
sol = get_greedy_solution(spec)
assert is_solution_feasible(spec, sol)
objective_function(spec, sol)

1125955.4000000004